In [ ]:
#MODELO PREDITIVO INDIVIDUAL PARA O TABU SEARCH E PARTICLE SWARM OPTIMIZATION - MODELO DE REGRESSÃO DE AFINAÇÃO DOS PARÂMETROS ---------------------------

import pandas as pd
import os
import random
import math
import numpy as np 
from matplotlib import pyplot as plt
import xlwings as xw
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

#EXCEL - CAMINHO DO FICHEIRO - COLOCAR DATASET RELATIVO AO TABU SEARCH OU PARTICLE SWARM OPTIMIZATION (ESTÁ COM FICHEIRO DE PARTICLE SWARM OPTIMIZATION)
#NOTA: O MODELO INDIVIDUAL REALIZADO SERÁ PARA O PARTICLE SWARM OPTIMIZATION, SE QUISER O TABU SEACH MUDAR O FICHEIRO EXCEL PARA DataSet_Regressão_TS
file_path = r"D:\Documentos\...\DataSet_Regressão_PSO.xlsx"
dados_problema= pd.read_excel(file_path)
print (dados_problema)


      MH  NJOBS  NMACHINES  CMAXOPT OBJFUNC  CMAX  TIMEEXEC  NUMPART  NUMITERA
0    PSO      6          6       55    Cmax    59      0.24       15       500
1    PSO      6          6       55    Cmax    59      0.28       15       536
2    PSO      6          6       55    Cmax    60      0.25       15       484
3    PSO      6          6       55    Cmax    60      0.25       15       437
4    PSO     10         10      887      WT  1455      1.25       25      1000
..   ...    ...        ...      ...     ...   ...       ...      ...       ...
127  PSO     30         10     1784      WT  2158     11.56       36      1934
128  PSO     30         10     1784      WT  2187     12.33       36      2121
129  PSO     50         10     2852      WT  2968     77.76       76      3025
130  PSO     50         10     2924      WT  3153     78.53       76      3030
131  PSO     50         10     2843      WT  3458     44.04      100      4000

[132 rows x 9 columns]


In [ ]:
#DIVIDIR DADOS EM TREINO E TESTE (RETIRAR COLONA CMAX, MH E TIMEEXEC (NÃO É UMA FEATURE))
dados_problema=dados_problema.drop(columns=['CMAX'])
dados_problema=dados_problema.drop(columns=['MH'])
dados_problema=dados_problema.drop(columns=['TIMEEXEC'])
#print (dados_problema)

#VARIÁVEIS INDEPENDENTES
X = dados_problema.drop(columns=['NUMPART', 'NUMITERA'])      #PARA TABU SEARCH É X = dados_problema.drop(columns=['TABULISTLEN', 'TS_STOPCRIT'])
#VARIÁVEL DEPENDENTE
y = dados_problema[['NUMPART', 'NUMITERA']]          #PARA TABU SEARCH É y = dados_problema[['TABULISTLEN', 'TS_STOPCRIT']] 

#COLUNA OBJFUNX: PASSAR CMAX = 0 E WT=1 - NECESSÁRIO EM ALGUMAS TÉCNICAS
X['OBJFUNC'] = X['OBJFUNC'].map({'Cmax': 0, 'WT': 1})

#DIVIDIR OS DADOS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#ÁRVORE DE DECISÃO ------------------------------------------------------------------------------------------------------------------------------

#FUNÇÃO PARA CALCULAR MÉTRICAS DE AVALIAÇÃO
def evaluate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    mape = np.mean(np.abs((y_true - y_pred) / y_true), axis=0) * 100
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    return mse, rmse, mae, mape, r2

#CRIAR MODELO ÁRVORE DE DESIÇÃO
base_model = DecisionTreeRegressor(random_state=42)
model = MultiOutputRegressor(base_model)

# DEFINIIR PARÂMETROS
param_grid = {
    'estimator__criterion': ['squared_error', 'absolute_error'],
    'estimator__max_depth': [3, 5, 10, 15, 20, None],
    'estimator__min_samples_split': [2, 5, 10, 15],
    'estimator__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 10, 15]
}

#GRIDSEARCH COM VALIDAÇÃO CRUZADA 
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

#TREINAR O MODELO À PROCURA DOS MELHORES HIPERPARÂMETROS
grid_search.fit(X_train, y_train)

#OBTER MELHOR MODELO
best_model = grid_search.best_estimator_
print("\nMelhores Hiperparâmetros:", grid_search.best_params_)

#PREVISÃO E CALCULO DAS MÉTRICAS DE AVALIAÇÃO
y_test_pred = best_model.predict(X_test)
mse_test, rmse_test, mae_test, mape_test, r2_test = evaluate_metrics(y_test, y_test_pred)


'''
#RESULTADOS DAS MÉTRICAS DE AVALIAÇÃO DE CADA VARIÁVEL DEPENDENTE
print("=== Resultados no CONJUNTO DE TESTE ===")
for i, target in enumerate(['NUMPART', 'NUMITERA']):        #PARA TABU SEARCH É (['TABULISTLEN', 'TS_STOPCRIT'])
    print(f"\nVariável: {target}")
    print(f"MSE: {mse_test[i]:.4f}")
    print(f"RMSE: {rmse_test[i]:.4f}")
    print(f"MAE: {mae_test[i]:.4f}")
    print(f"MAPE: {mape_test[i]:.2f}%")
    print(f"R²: {r2_test[i]:.4f}")
'''

#RESULTADOS GERAIS - MÉDIA DOS RESULTADOS DAS DUAS VARIÁVEIS (À EXCEÇÃO DO RMSE) 
mse_test_mean = np.mean(mse_test)
rmse_test_mean = np.sqrt(mse_test_mean)
mae_test_mean = np.mean(mae_test)
mape_test_mean = np.mean(mape_test)
r2_test_mean = np.mean(r2_test)

print("\n=== RESULTADO GERAL (MÉDIA TESTE) ===")
print(f"MSE: {mse_test_mean:.4f}")
print(f"RMSE: {rmse_test_mean:.4f}")
print(f"MAE: {mae_test_mean:.4f}")
print(f"MAPE: {mape_test_mean:.2f}%")
print(f"R²: {r2_test_mean:.4f}")


#IMPORTANCIA DAS FEATURES
#CALCULAR A MÉDIA DA IMPORTÂNCIA DE CADA VARIAVEL DEPENDENTE (CADA ÁRVORE DE DECISÃO)
importances_raw = np.mean([est.feature_importances_ for est in best_model.estimators_], axis=0)
importances = importances_raw / importances_raw.sum()
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
importance_df['Importance'] = importance_df['Importance'].round(4)
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\n=== Importância das Features (Média entre os modelos, normalizada) ===")
print(importance_df)



Melhores Hiperparâmetros: {'estimator__criterion': 'squared_error', 'estimator__max_depth': 3, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 10}

=== RESULTADO GERAL (MÉDIA TESTE) ===
MSE: 31196.1483
RMSE: 176.6243
MAE: 69.5098
MAPE: 8.16%
R²: 0.6452

=== Importância das Features (Média entre os modelos, normalizada) ===
     Feature  Importance
0      NJOBS      0.9874
2    CMAXOPT      0.0066
3    OBJFUNC      0.0060
1  NMACHINES      0.0000


'\n#COMPARAÇÃO ENTRE TREINO E TESTE\ncomparison_df = pd.DataFrame({\n    \'Variável\': [\'NJOBS\', \'NMACHINES\', \'MÉDIA GERAL\'],\n    \'MSE Treino\': list(mse_train) + [np.mean(mse_train)],\n    \'MSE Teste\': list(mse_test) + [mse_test_mean],\n    \'RMSE Treino\': list(rmse_train) + [np.mean(rmse_train)],\n    \'RMSE Teste\': list(rmse_test) + [rmse_test_mean],\n    \'MAE Treino\': list(mae_train) + [np.mean(mae_train)],\n    \'MAE Teste\': list(mae_test) + [mae_test_mean],\n    \'MAPE Treino\': list(mape_train) + [np.mean(mape_train)],\n    \'MAPE Teste\': list(mape_test) + [mape_test_mean],\n    \'R² Treino\': list(r2_train) + [np.mean(r2_train)],\n    \'R² Teste\': list(r2_test) + [r2_test_mean]\n})\n\nprint("\n=== QUADRO COMPARATIVO TREINO vs TESTE ===")\nprint(comparison_df.round(4))\n'

In [ ]:
#RANDOM FOREST ------------------------------------------------------------------------------------------------------------------------------------------

#FUNÇÃO PARA CALCULAR AS MÉTRICAS DE AVALIAÇÃO
def evaluate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    mape = np.mean(np.abs((y_true - y_pred) / y_true), axis=0) * 100
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    return mse, rmse, mae, mape, r2

#CRIAR MODELO RANDOM FOREST
base_model = RandomForestRegressor(random_state=42)
model = MultiOutputRegressor(base_model)

#DEFINIR PARÂMETROS
param_grid = {
    'estimator__n_estimators': [50, 100, 200, 300, 400],
    'estimator__max_depth': [None, 3, 5, 7, 10, 20, 30],
    'estimator__min_samples_split': [2, 5, 10, 15],
    'estimator__min_samples_leaf': [1, 2, 4, 6],
    'estimator__max_features': [None, 'sqrt', 'log2'],
}

#GRIDSEACH COM VALIDAÇÃO CRUZADAS
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

#TREINAR O MODELO À PROCURA DOS MELHORES HIPERPARÂMETROS
grid_search.fit(X_train, y_train)

#OBTER O MELHOR MODELO 
best_model = grid_search.best_estimator_
print("\nMelhores Hiperparâmetros:", grid_search.best_params_)

#PREVISÃO E CALCULO DAS MÉTRICAS DE AVALIAÇÃO
y_test_pred = best_model.predict(X_test)
mse_test, rmse_test, mae_test, mape_test, r2_test = evaluate_metrics(y_test, y_test_pred)

'''
#RESULTADOS DAS MÉTRICAS DE AVALIAÇÃO DE CADA VARIÁVEL DEPENDENTE
print("=== Resultados no CONJUNTO DE TESTE (Random Forest) ===")
for i, target in enumerate(['NUMPART', 'NUMITERA']):             #PARA TABU SEARCH É (['TABULISTLEN', 'TS_STOPCRIT'])
    print(f"\nVariável: {target}")
    print(f"MSE: {mse_test[i]:.4f}")
    print(f"RMSE: {rmse_test[i]:.4f}")
    print(f"MAE: {mae_test[i]:.4f}")
    print(f"MAPE: {mape_test[i]:.2f}%")
    print(f"R²: {r2_test[i]:.4f}")
'''

#RESULTADOS GERAIS - MÉDIA DOS RESULTADOS DAS DUAS VARIÁVEIS (À EXCEÇÃO DO RMSE)
mse_test_mean = np.mean(mse_test)
rmse_test_mean = np.sqrt(mse_test_mean)
mae_test_mean = np.mean(mae_test)
mape_test_mean = np.mean(mape_test)
r2_test_mean = np.mean(r2_test)

print("\n=== RESULTADO GERAL (MÉDIA TESTE) ===")
print(f"MSE: {mse_test_mean:.4f}")
print(f"RMSE: {rmse_test_mean:.4f}")
print(f"MAE: {mae_test_mean:.4f}")
print(f"MAPE: {mape_test_mean:.2f}%")
print(f"R²: {r2_test_mean:.4f}")

#IMPORTANCIA DAS FEATURES
#CALCULAR A MÉDIA DA IMPORTÂNCIA DE CADA VARIAVEL DEPENDENTE 
importances = np.mean([est.feature_importances_ for est in best_model.estimators_], axis=0)
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
importance_df['Importance'] = importance_df['Importance'].round(4)
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\n=== Importância das Features ===")
print(importance_df)



Melhores Hiperparâmetros: {'estimator__max_depth': 3, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 5, 'estimator__n_estimators': 50}

=== RESULTADO GERAL (MÉDIA TESTE) ===
MSE: 29454.3671
RMSE: 171.6227
MAE: 67.1109
MAPE: 7.75%
R²: 0.6915

=== Importância das Features ===
     Feature  Importance
0      NJOBS      0.5975
2    CMAXOPT      0.3928
3    OBJFUNC      0.0094
1  NMACHINES      0.0003


'\n# Comparação Treino x Teste (tabela resumo)\ncomparison_df = pd.DataFrame({\n    \'Variável\': [\'NJOBS\', \'NMACHINES\', \'MÉDIA GERAL\'],\n    \'MSE Treino\': list(mse_train) + [np.mean(mse_train)],\n    \'MSE Teste\': list(mse_test) + [mse_test_mean],\n    \'RMSE Treino\': list(rmse_train) + [np.mean(rmse_train)],\n    \'RMSE Teste\': list(rmse_test) + [rmse_test_mean],\n    \'MAE Treino\': list(mae_train) + [np.mean(mae_train)],\n    \'MAE Teste\': list(mae_test) + [mae_test_mean],\n    \'MAPE Treino\': list(mape_train) + [np.mean(mape_train)],\n    \'MAPE Teste\': list(mape_test) + [mape_test_mean],\n    \'R² Treino\': list(r2_train) + [np.mean(r2_train)],\n    \'R² Teste\': list(r2_test) + [r2_test_mean]\n})\n\nprint("\n=== QUADRO COMPARATIVO TREINO vs TESTE (Random Forest) ===")\nprint(comparison_df.round(4))\n'

In [ ]:
#REGRESSÃO LINEAR ----------------------------------------------------------------------------------------------------------

#FUNÇÃO PARA CALCULAR AS MÉTRICAS DE AVALIAÇÃO
def evaluate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    mape = np.mean(np.abs((y_true - y_pred) / y_true), axis=0) * 100
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    return mse, rmse, mae, mape, r2

#CRIAR MODELO DE REGRESSÃO LINEAR
base_model = LinearRegression()
model = MultiOutputRegressor(base_model)
model.fit(X_train, y_train)

#PREVISÕES
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#CALCULAR MÉTRICAS DE AVALIAÇÃO
mse_test, rmse_test, mae_test, mape_test, r2_test = evaluate_metrics(y_test, y_test_pred)

'''
#RESULTADOS DAS MÉTRICAS DE AVALIAÇÃO DE CADA VARIÁVEL DEPENDENTE
print("=== Resultados no CONJUNTO DE TESTE ===")
for i, target in enumerate(['TABULISTLEN', 'TS_STOPCRIT']):     #PARA TABU SEARCH É (['TABULISTLEN', 'TS_STOPCRIT'])
    print(f"\nVariável: {target}")
    print(f"MSE: {mse_test[i]:.4f}")
    print(f"RMSE: {rmse_test[i]:.4f}")
    print(f"MAE: {mae_test[i]:.4f}")
    print(f"MAPE: {mape_test[i]:.2f}%")
    print(f"R²: {r2_test[i]:.4f}")
'''

#RESULTADOS GERAIS - MÉDIA DOS RESULTADOS DAS DUAS VARIÁVEIS (À EXCEÇÃO DO RMSE)
mse_test_mean = np.mean(mse_test)
rmse_test_mean =  np.sqrt(mse_test_mean)
mae_test_mean = np.mean(mae_test)
mape_test_mean = np.mean(mape_test)
r2_test_mean = np.mean(r2_test)

print("\n=== RESULTADO GERAL (MÉDIA TESTE) ===")
print(f"MSE: {mse_test_mean:.4f}")
print(f"RMSE: {rmse_test_mean:.4f}")
print(f"MAE: {mae_test_mean:.4f}")
print(f"MAPE: {mape_test_mean:.2f}%")
print(f"R²: {r2_test_mean:.4f}")



=== RESULTADO GERAL (MÉDIA TESTE) ===
MSE: 41052.8317
RMSE: 202.6150
MAE: 117.4866
MAPE: 13.96%
R²: 0.5558


'\n# Comparação Treino x Teste (tabela resumo)\ncomparison_df = pd.DataFrame({\n    \'Variável\': [\'NJOBS\', \'NMACHINES\', \'MÉDIA GERAL\'],\n    \'MSE Treino\': list(mse_train) + [np.mean(mse_train)],\n    \'MSE Teste\': list(mse_test) + [mse_test_mean],\n    \'RMSE Treino\': list(rmse_train) + [np.mean(rmse_train)],\n    \'RMSE Teste\': list(rmse_test) + [rmse_test_mean],\n    \'MAE Treino\': list(mae_train) + [np.mean(mae_train)],\n    \'MAE Teste\': list(mae_test) + [mae_test_mean],\n    \'MAPE Treino\': list(mape_train) + [np.mean(mape_train)],\n    \'MAPE Teste\': list(mape_test) + [mape_test_mean],\n    \'R² Treino\': list(r2_train) + [np.mean(r2_train)],\n    \'R² Teste\': list(r2_test) + [r2_test_mean]\n})\n\nprint("\n=== QUADRO COMPARATIVO TREINO vs TESTE (Regressão Linear Simples) ===")\nprint(comparison_df.round(4))'